<a href="https://colab.research.google.com/github/SKgain/dl-model/blob/main/final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/JavaFest/dataset_split.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1066.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1067.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1069.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_107.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1070.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1071.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1073.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1076.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1077.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1078.jpg  
  inflating: /content/dataset_split/train/General_Waste/General_Waste_1079.jpg  
  inflating: /content/dataset_split/train/General_Waste/Gen

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# --- Paths and Constants ---
train_dir = '/content/dataset_split/train'
val_dir = '/content/dataset_split/val'
test_dir = '/content/dataset_split/test'

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64
NUM_CLASSES = 4
BEST_MODEL_PATH = 'waste_classifier_final.keras' # Using the recommended .keras format

# --- Data Preparation ---
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)
test_val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)
val_generator = test_val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
test_generator = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# --- PHASE 1: INITIAL TRAINING ---
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("🚀 Starting Phase 1: Initial Training...")
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# --- PHASE 2: FINE-TUNING ---

print("\n🚀 Starting Phase 2: Fine-Tuning...")
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # Use a very low learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# --- Define Callbacks for Optimization ---
# Stop training when validation loss stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=4, # Wait 4 epochs before stopping
    restore_best_weights=True # Automatically restore the weights from the best epoch
)
# Save only the best model based on validation accuracy
model_checkpoint = ModelCheckpoint(
    filepath=BEST_MODEL_PATH,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

# --- Continue Training with Fine-Tuning and Callbacks ---
fine_tune_epochs = 20 # Set a max number of epochs
total_epochs = 10 + fine_tune_epochs

history_fine_tune = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=val_generator,
    callbacks=[early_stopping, model_checkpoint] # Add the callbacks here
)

# --- FINAL EVALUATION ---
# The best model is already saved by ModelCheckpoint, and EarlyStopping restored the best weights
print("\nEvaluating the best fine-tuned model on the test set...")
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n✅ Final Test Accuracy of the Best Model: {test_acc*100:.2f}%")

print(f"\nBest model saved to {BEST_MODEL_PATH}")

Found 5250 images belonging to 4 classes.
Found 1124 images belonging to 4 classes.
Found 1128 images belonging to 4 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
🚀 Starting Phase 1: Initial Training...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 89s 926ms/step - accuracy: 0.6350 - loss: 0.8951 - val_accuracy: 0.8870 - val_loss: 0.3033
Epoch 2/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 68s 815ms/step - accuracy: 0.8677 - loss: 0.3518 - val_accuracy: 0.9128 - val_loss: 0.2505
Epoch 3/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 68s 819ms/step - accuracy: 0.8882 - loss: 0.2906 - val_accuracy: 0.9217 - val_loss: 0.2321
Epoch 4/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 81s 811ms/step - accuracy: 0.9102 - loss: 0.2383 - val_accuracy: 0.9262 - val_loss: 0.2172
Epoch 5/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 67s 807ms/step - accuracy: 0.9147 - loss: 0.2271 - val_accuracy: 0.9244 - val_loss: 0.2057
Epoch 6/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 68s 818ms/step - accuracy: 0.9157 - loss: 0.2223 - val_accuracy: 0.9244 - val_loss: 0.2065
Epoch 7/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 68s 820ms/step - accuracy: 0.9169 - loss: 0.2110 - val_accuracy: 0.9262 - val_loss: 0.2000
Epoch 8/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 68s 816ms/step - accuracy: 0.9221 - loss: 0.2031 - val_accu

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import files
import os
import zipfile

# --- Configuration ---
BEST_MODEL_PATH = 'waste_classifier_final.keras'
TEST_DATA_DIR = '/content/dataset_split/test'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64

# --- 1. Generate All Reports ---

print("--- Generating Reports ---")

# Load the best model that was saved
model = tf.keras.models.load_model(BEST_MODEL_PATH)

# Prepare the test data generator
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Get true labels and predictions
true_labels = test_generator.classes
class_names = list(test_generator.class_indices.keys())
pred_probs = model.predict(test_generator)
pred_labels = np.argmax(pred_probs, axis=1)

# Save Classification Report
report = classification_report(true_labels, pred_labels, target_names=class_names)
with open("classification_report.txt", "w") as f:
    f.write(report)
print("✅ Classification Report saved.")

# Save Confusion Matrix
cm = confusion_matrix(true_labels, pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig("confusion_matrix.png")
print("✅ Confusion Matrix image saved.")
plt.close() # Close the plot to avoid displaying it here

# Save Final Accuracy
test_loss, test_acc = model.evaluate(test_generator)
summary_text = f"Final Test Accuracy: {test_acc*100:.2f}%\n"
summary_text += f"Final Test Loss: {test_loss:.4f}"
with open("accuracy_summary.txt", "w") as f:
    f.write(summary_text)
print("✅ Accuracy summary saved.")

# --- 2. Zip All Files Together ---

print("\n--- Zipping all files for download ---")
zip_filename = 'sortify_model_and_reports.zip'
files_to_zip = [
    BEST_MODEL_PATH,
    'classification_report.txt',
    'confusion_matrix.png',
    'accuracy_summary.txt'
]

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        if os.path.exists(file):
            zipf.write(file)
            print(f"- Added {file} to zip.")
        else:
            print(f"- WARNING: Could not find {file} to add to zip.")

# --- 3. Download the Zip File ---

print(f"\n🚀 Downloading {zip_filename}... Please wait.")
files.download(zip_filename)

--- Generating Reports ---
Found 1128 images belonging to 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 301ms/step
✅ Classification Report saved.
✅ Confusion Matrix image saved.
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 243ms/step - accuracy: 0.9604 - loss: 0.1080
✅ Accuracy summary saved.

--- Zipping all files for download ---
- Added waste_classifier_final.keras to zip.
- Added classification_report.txt to zip.
- Added confusion_matrix.png to zip.
- Added accuracy_summary.txt to zip.

🚀 Downloading sortify_model_and_reports.zip... Please wait.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
from google.colab import files

# This will open a dialog to let you upload a file
uploaded = files.upload()

# Get the name of the file you just uploaded
test_image_path = list(uploaded.keys())[0]

print(f"Image '{test_image_path}' uploaded and is ready for testing.")

Saving r5.jpg to r5 (1).jpg
Image 'r5 (1).jpg' uploaded and is ready for testing.


In [20]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# --- Configuration ---
# Make sure this is the name of your best saved model
MODEL_PATH = 'waste_classifier_final.keras'
IMAGE_SIZE = (224, 224) # Must be the same size you trained with
CLASS_NAMES = ['Compost', 'General_Waste', 'Hazardoos', 'Recycle']


# --- 1. Load the trained model ---
print(f"Loading model: {MODEL_PATH}")
model = tf.keras.models.load_model(MODEL_PATH)


# --- 2. Load and Preprocess the single image ---
# The 'test_image_path' variable is from the upload cell above
img = image.load_img(test_image_path, target_size=IMAGE_SIZE)

# Convert the image to a NumPy array
img_array = image.img_to_array(img)

# Add an extra dimension for the batch (model expects a batch of images)
img_array = np.expand_dims(img_array, axis=0)

# Normalize the image (the same way you did for training)
img_array /= 255.0


# --- 3. Make and interpret the prediction ---
# Get the model's prediction
predictions = model.predict(img_array)

# Find the class with the highest probability
predicted_class_index = np.argmax(predictions[0])
predicted_class_name = CLASS_NAMES[predicted_class_index]
confidence = np.max(predictions[0]) * 100


# --- 4. Print the final result ---
print("\n========================================")
print(f"Prediction: {predicted_class_name}")
print(f"Confidence: {confidence:.2f}%")
print("========================================")

Loading model: waste_classifier_final.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

Prediction: Recycle
Confidence: 99.96%
